In [1]:
# add all the imports
from __future__ import print_function

import argparse
from datetime import datetime
import os
import sys
import time

import tensorflow as tf
import numpy as np

In [2]:
# import models from deeplab_resnet
from deeplab_resnet import DeepLabResNetModel, ImageReader, decode_labels, inv_preprocess, prepare_label

In [3]:
# some parameteres setting:
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)

BATCH_SIZE = 2 #10
DATA_DIRECTORY = '/data/VOC_dataset/voc2012_trainval'
DATA_LIST_PATH = './dataset/train.txt'
IGNORE_LABEL = 255
INPUT_SIZE = '321,321'
LEARNING_RATE = 2.5e-4
MOMENTUM = 0.9
NUM_CLASSES = 21
NUM_STEPS = 20001
POWER = 0.9
RANDOM_SEED = 1234
RESTORE_FROM = './deeplab_resnet.ckpt'
SAVE_NUM_IMAGES = 2
SAVE_PRED_EVERY = 1000
SNAPSHOT_DIR = './snapshots/'
WEIGHT_DECAY = 0.0005

### Create the model and start the training...

In [4]:
# setting
input_size = (321,321)
tf.set_random_seed(RANDOM_SEED)

In [5]:
# Create queue coordinator.
coord = tf.train.Coordinator()

In [6]:
with tf.name_scope("create_inputs"):
    reader = ImageReader(
        DATA_DIRECTORY,
        DATA_LIST_PATH,
        input_size,
        True,
        True,
        IGNORE_LABEL,
        IMG_MEAN,
        coord)
    image_batch, label_batch = reader.dequeue(BATCH_SIZE)

In [7]:
print(image_batch.shape)

(2, 321, 321, 3)


In [10]:
print(type(image_batch))

<class 'tensorflow.python.framework.ops.Tensor'>


In [11]:
print(label_batch.shape)

(2, 321, 321, 1)


In [14]:
# Create network.
net = DeepLabResNetModel({'data': image_batch}, is_training=True, num_classes=NUM_CLASSES)
# For a small batch size, it is better to keep 
# the statistics of the BN layers (running means and variances)
# frozen, and to not update the values provided by the pre-trained model. 
# If is_training=True, the statistics will be updated during the training.
# Note that is_training=False still updates BN parameters gamma (scale) and beta (offset)
# if they are presented in var_list of the optimiser definition.

In [ ]:
# Predictions.
raw_output = net.layers['fc1_voc12']
# Which variables to load. Running means and variances are not trainable,
# thus all_variables() should be restored.
restore_var = [v for v in tf.global_variables() if 'fc' not in v.name or not args.not_restore_last]
all_trainable = [v for v in tf.trainable_variables() if 'beta' not in v.name and 'gamma' not in v.name]
fc_trainable = [v for v in all_trainable if 'fc' in v.name]
conv_trainable = [v for v in all_trainable if 'fc' not in v.name] # lr * 1.0
fc_w_trainable = [v for v in fc_trainable if 'weights' in v.name] # lr * 10.0
fc_b_trainable = [v for v in fc_trainable if 'biases' in v.name] # lr * 20.0
assert(len(all_trainable) == len(fc_trainable) + len(conv_trainable))
assert(len(fc_trainable) == len(fc_w_trainable) + len(fc_b_trainable))